## Exercise D Developer Test
1. Please import mseed file
2. Build a database file with proper normal form and constraints\
3. Import data into the newly built database
3. Build A visualization for the data